Target:

Basic Network. 
Results:

16k Parameters

Best Train Accuracy: 99.26

Best Test Accuracy: 98.92

Observations:

1. Model has decent parameters
2. Overfitting can be seen happening after epoch 8

In [ ]:

from __future__ import print_function
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import itertools
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(1, 8, 3))
        self.conv2= nn.Sequential(nn.Conv2d(8, 8, 3))
        self.conv3= nn.Sequential(nn.MaxPool2d(2, 2),nn.Conv2d(8, 16, 1))
        self.conv4 = nn.Sequential(nn.Conv2d(16, 16, 3))
        self.conv5 = nn.Sequential(nn.Conv2d(16, 16, 3))
        self.conv6 = nn.Sequential(nn.MaxPool2d(2, 2),nn.Conv2d(16, 32, 1))
        self.conv7 = nn.Sequential(nn.Conv2d(32, 32, 3))
        self.conv8= nn.Sequential(nn.Conv2d(32, 10, 2))
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))
        x = F.relu(self.conv7(x))
        x = self.conv8(x)

        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [ ]:
#!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              80
            Conv2d-2            [-1, 8, 24, 24]             584
         MaxPool2d-3            [-1, 8, 12, 12]               0
            Conv2d-4           [-1, 16, 12, 12]             144
            Conv2d-5           [-1, 16, 10, 10]           2,320
            Conv2d-6             [-1, 16, 8, 8]           2,320
         MaxPool2d-7             [-1, 16, 4, 4]               0
            Conv2d-8             [-1, 32, 4, 4]             544
            Conv2d-9             [-1, 32, 2, 2]           9,248
           Conv2d-10             [-1, 10, 1, 1]           1,290
Total params: 16,530
Trainable params: 16,530
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.13
Params size (MB): 0.06
Estimated Tot

In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation(5),      
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    train_correct = 0
    loss = 0
    model.train()
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True) 
        train_correct += pred.eq(target.view_as(pred)).sum().item()

    loss /= len(test_loader.dataset)

    print('\nTrain set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
    loss, train_correct, len(train_loader.dataset),
    100. * train_correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  
            pred = output.argmax(dim=1, keepdim=True) 
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
no_of_epochs = 15
for epoch in range(no_of_epochs):
    print("Epoch Number :=> ",epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch Number :=>  1

Train set: Average loss: 0.0000, Accuracy: 31403/60000 (52.34%)


Test set: Average loss: 0.1623, Accuracy: 9495/10000 (94.95%)

Epoch Number :=>  2

Train set: Average loss: 0.0000, Accuracy: 57450/60000 (95.75%)


Test set: Average loss: 0.0811, Accuracy: 9750/10000 (97.50%)

Epoch Number :=>  3

Train set: Average loss: 0.0000, Accuracy: 58356/60000 (97.26%)


Test set: Average loss: 0.0688, Accuracy: 9776/10000 (97.76%)

Epoch Number :=>  4

Train set: Average loss: 0.0000, Accuracy: 58696/60000 (97.83%)


Test set: Average loss: 0.0543, Accuracy: 9819/10000 (98.19%)

Epoch Number :=>  5

Train set: Average loss: 0.0000, Accuracy: 58846/60000 (98.08%)


Test set: Average loss: 0.0538, Accuracy: 9831/10000 (98.31%)

Epoch Number :=>  6

Train set: Average loss: 0.0000, Accuracy: 59023/60000 (98.37%)


Test set: Average loss: 0.0391, Accuracy: 9871/10000 (98.71%)

Epoch Number :=>  7

Train set: Average loss: 0.0000, Accuracy: 59110/60000 (98.52%)


Test set: Ave